In [1]:
import jax.numpy as jnp
from jax import grad, vmap, jit, random, value_and_grad,vjp
import jax.nn as nn
import tqdm

from scipy.ndimage.interpolation import shift


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


### RNN using Jax?


In [2]:
import numpy as np
import haiku as hk
import jax
import optax

In [3]:
def mso(samples,horizon=12):
    t=jnp.arange(0,samples)
    X=jnp.sin(0.2*t)+jnp.sin(0.311*t)+jnp.sin(0.42*t)+jnp.sin(0.51*t)
    y=shift(X,horizon)
    return X,y

In [ ]:
X,y=mso(25000)

In [ ]:
def rnn_forward(x_t,h_prev,hidden_size=32):
    f=hk.VanillaRNN(hidden_size=hidden_size)
    h_t,h_t=f(x_t,h_prev)
    g=hk.Linear(1)
    y_t=nn.relu(g(h_t))
    return y_t,h_t

In [ ]:
rnn_forward_pure=hk.without_apply_rng(hk.transform(rnn_forward))

In [ ]:
key=random.PRNGKey(0)
params=rnn_forward_pure.init(key,jnp.array([1.0]),jnp.array([1.0]*32))

In [ ]:
def naive_loss_fn(params,X,y,t):
    """
    X: input timeseries
    y: output timeseries
    t: timestep to calculate the loss
    """
    h_t=jnp.array([0.0]*32)
    for i in range(t):
        x_t=X[np.array([t])]
        y_t_pred,h_t=rnn_forward_pure.apply(params,x_t,h_t)
    y_t=y[np.array([t-12])]
    mse_loss=((y_t_pred-y_t)**2).sum()
    return mse_loss


In [ ]:
optimizer = optax.adam(0.01)
opt_state = optimizer.init(params)

In [ ]:
#Lets design our training loop
h_t=jnp.array([0.0]*32) #The initial state of the RNN
for t in tqdm.tqdm(range(X.shape[0])):
    x_t=X[np.array([t])]
    if t>=12: 
        loss,dy_dparam=value_and_grad(naive_loss_fn)(params,X,y,t)
        updates, opt_state = optimizer.update(dy_dparam, opt_state)
        params=optax.apply_updates(params,updates)
        print(loss.block_until_ready())

### Problems with the naive implementation?
- We need to unroll the entirity of input until the time step 0 to calculate loss at time t
- Gradient calculation time increases as t increases
- Batch implementation is tricky as we cant parallize this array 

Properties we want to have our P_RNN to have:
- forward function that takes params, hidden_state, input -> hidden_state
- sensitivity function that takes params, hidden_state, p - (input,hidden_state) trajectory

In [ ]:
def rnn_forward(i_t,h_tminus1,hidden_size=32):
    f=hk.VanillaRNN(hidden_size=hidden_size)
    h_t,h_t=f(i_t,h_tminus1)
    return h_t

In [ ]:
rnn_forward_pure=hk.without_apply_rng(hk.transform(rnn_forward))
key=random.PRNGKey(0)
rnn_params=rnn_forward_pure.init(key,jnp.array([1.0]),jnp.array([1.0]*32))
rnn_forward_pure_jit=jit(rnn_forward_pure.apply)

In [ ]:
def sensitivity(rnn_params,hidden_state,rnn_trajectory):
    inputs,hidden_states=rnn_trajectory[0],rnn_trajectory[1]
    rnn_jac_theta=jax.jacrev(rnn_forward_pure.apply)
    rnn_jac_hidden=jax.jacrev(rnn_forward_pure.apply,argnums=2)
    del_h_tminus1_theta=rnn_jac_theta(rnn_params,inputs[0],hidden_states[0])
    del_h_t_theta=del_h_tminus1_theta
    
    def sensitivity_calc(del_h_tminus1_theta,trajectory):
        i_t,h_tminus1=trajectory
        del_f_theta=rnn_jac_theta(rnn_params,i_t,h_tminus1)
        del_f_h_tminus1=rnn_jac_hidden(rnn_params,i_t,h_tminus1)
        # del_h_t_theta=del_f_h_tminus1*del_h_tminus1_theta
        del_h_t_theta= jax.tree_map(lambda x: jnp.tensordot(del_f_h_tminus1,x,axes=1), del_h_tminus1_theta) 
        # del_h_t_theta+=del_f_theta
        del_h_t_theta=jax.tree_multimap(lambda x, y: x+y, del_h_t_theta, del_f_theta)
        return del_h_t_theta,None
    
    if len(rnn_trajectory)>1:
        del_h_t_theta,_=jax.lax.scan(sensitivity_calc,del_h_tminus1_theta,(inputs[1:],hidden_states[1:]))
    return del_h_t_theta
sensitivity=jit(sensitivity)

In [ ]:
truncation=10

In [ ]:
def g(h_t):
    out=nn.relu(hk.Linear(8)(h_t))
    out=hk.Linear(1)(out)
    return out

g=hk.without_apply_rng(hk.transform(g))
g_params=g.init(key,jnp.array([1.0]*32))

def loss_fn(g_params,h_t,y_t):
    y_t_pred=g.apply(g_params,h_t)
    return ((y_t_pred-y_t)**2).sum()

loss_fn=jit(loss_fn)

In [ ]:
optimizer_rnn = optax.adam(0.0001)
opt_state_rnn = optimizer_rnn.init(rnn_params)
optimizer_g = optax.adam(0.001)
opt_state_g = optimizer_g.init(g_params)

In [ ]:
h_tminus1=jnp.array([0.0]*32) #The initial state of the RNN
trajectories=[jnp.zeros((truncation,1)),jnp.zeros((truncation,32))]

def update_trajectories(trajectories,i,h):
    trajectories[0]=jnp.concatenate((trajectories[0][1:],i.reshape(1,-1)),axis=0)
    trajectories[1]=jnp.concatenate((trajectories[1][1:],h.reshape(1,-1)),axis=0)
    return trajectories
    
losses=[]
for epoch in range(10):
    for t in tqdm.tqdm(range(X.shape[0])):
        x_t,y_t=X[np.array([t])],y[np.array([t])]
        #Update trajectories
        trajectories=update_trajectories(trajectories,x_t,h_tminus1)
        h_t=rnn_forward_pure_jit(rnn_params,x_t,h_tminus1)
        #Predict the output and get the loss
        loss,grad_g_params=value_and_grad(loss_fn)(g_params,h_t,y_t)
        #Calculate the gradient wrt to rnn_params using the sensitivities of the rnn and g layers
        rnn_sensitivity_t=sensitivity(rnn_params,h_t,trajectories)
        g_sensitivity_t=jax.jacfwd(loss_fn,argnums=1)(g_params,h_t,y_t)
        grad_rnn_params=jax.tree_multimap(lambda x: jnp.tensordot(g_sensitivity_t,x,axes=1), rnn_sensitivity_t) 
        #Update the gradients
        updates, opt_state_rnn = optimizer_rnn.update(grad_rnn_params, opt_state_rnn)
        rnn_params=optax.apply_updates(rnn_params,updates)
        updates, opt_state_g = optimizer_g.update(grad_g_params, opt_state_g)
        g_params=optax.apply_updates(g_params,updates)
        #Calculate average loss and print if it is print frequency
        losses.append(loss)
        if len(losses)==100:
            print("Loss: ",np.mean(losses))
            losses=[]
        h_tminus1=h_t #Previous state becomes the current state before the next iteration

Awesome! It works, and its super fast too! Let's do some sanity check for the sensitivity

In [ ]:
sample_x=X[0:20]

h_t=jnp.array([0.0]*32)
def test_sensitivity(rnn_params,sample_x,h_t):
    for i in range(20):
        h_t=rnn_forward_pure_jit(rnn_params,sample_x[np.array([i])],h_t)
    return h_t

true_sensitivity=jax.jacrev(test_sensitivity)(rnn_params,sample_x,h_tminus1)

hidden_states=[]
for i in range(20):
    hidden_states.append(h_t)
    h_t=rnn_forward_pure_jit(rnn_params,sample_x[np.array([i])],h_t)
print("Hidden state calculated correctly: ",jnp.allclose(h_t,test_sensitivity(rnn_params,sample_x,h_tminus1)))
hidden_states=jnp.stack(hidden_states)
our_sensitivity=sensitivity(rnn_params,None,[sample_x.reshape(-1,1),hidden_states])
print("Sensitivity calculated correctly: ",jnp.allclose(true_sensitivity['vanilla_rnn/linear']['w'],our_sensitivity['vanilla_rnn/linear']['w'],atol=0.0001))

Next Step? create a haiku model that can be transformed, it automatically manages state and is differentiable with grad

Notes:
1. Maybe we should use haiku states
2. trajectories is updated in forward pass as a state variable
3. .apply is differentiable, and should use the trajectories to calculate sensitivities internally

In [4]:
import sys
sys.path.append('../')


In [5]:
from src.models import MultiplicativeRNN

In [6]:
def rnn_forward(obs,act,last_state):
    rnn=MultiplicativeRNN(10,4,32)
    out,state=rnn(obs,act,last_state)
    return out,state

In [7]:
rnn_forward_pure=hk.without_apply_rng(hk.transform(rnn_forward))


In [26]:
key=random.PRNGKey(0)
sample_o=jax.random.normal(key,[20,10])
sample_a=jax.random.normal(key,[20,4])

h_t=MultiplicativeRNN.initial_state(10,4,32,50)
rnn_params=rnn_forward_pure.init(key,sample_o[0],sample_a[0],h_t)
def test_sensitivity(rnn_params,sample_o,sample_a,h_t):
    for i in range(20):
        out,h_t=jit(rnn_forward_pure.apply)(rnn_params,sample_o[i],sample_a[i],h_t)
    return out

true_sensitivity=jax.jacrev(test_sensitivity)(rnn_params,sample_o,sample_a,h_t)

hidden_states=[]
for i in range(20):
    out,h_t=rnn_forward_pure.apply(rnn_params,sample_o[i],sample_a[i],h_t)
print("Hidden state calculated correctly: ",jnp.allclose(out,test_sensitivity(rnn_params,sample_o,sample_a,h_t)))
our_sensitivity=MultiplicativeRNN.sensitivity(rnn_forward_pure,rnn_params,h_t)
print("Sensitivity calculated correctly: ",jnp.allclose(true_sensitivity['multiplicative_rnn']['w_h'],
                                                        our_sensitivity['multiplicative_rnn']['w_h'],atol=0.0001))

Hidden state calculated correctly:  True
Sensitivity calculated correctly:  True


In [27]:
h_t['last_hidden_states']

(DeviceArray([0.05210083, 0.56093156, 0.31466708, 0.61934066, 0.49535763,
              0.35091436, 0.6882081 , 0.4596967 , 0.7190378 , 0.4332522 ,
              0.26202354, 0.83624905, 0.43509468, 0.79670703, 0.5144912 ,
              0.09377195, 0.812147  , 0.4114412 , 0.8281094 , 0.63258237,
              0.3755338 , 0.31899738, 0.35792035, 0.80044085, 0.04639092,
              0.58927727, 0.83882535, 0.16626808, 0.29229656, 0.06218464,
              0.78846854, 0.8522291 ], dtype=float32),
 TrajectoryState(last_hidden_states=DeviceArray([[0.        , 0.        , 0.        , ..., 0.        ,
               0.        , 0.        ],
              [0.        , 0.        , 0.        , ..., 0.        ,
               0.        , 0.        ],
              [0.        , 0.        , 0.        , ..., 0.        ,
               0.        , 0.        ],
              ...,
              [0.74492073, 0.18012244, 0.47929782, ..., 0.31637982,
               0.5986662 , 0.83822817],
              [

GOOD JOB!